# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:

# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data_cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,42.55,68,7,4.00,US,1702325191
1,1,carnarvon,-24.8667,113.6333,74.86,58,0,7.29,AU,1702325191
2,2,papatowai,-46.5619,169.4708,47.79,70,54,19.91,NZ,1702325192
3,3,lakheri,25.6667,76.1667,61.57,45,0,2.93,IN,1702325192
4,4,afaahiti,-17.7500,-149.2833,82.27,80,92,22.41,PF,1702325192


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

city_data_df_cleaned = city_data_df.dropna(subset=['Lng', 'Lat'])

# Configure the map plot
map_plot_1 = city_data_df_cleaned.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    size='Humidity',
    cmap='Viridis',
    colorbar=True,
    legend=True,
    hover_cols=['City'],
    color = ['City'],
    tiles='OSM')

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
max_temp = city_data_df[city_data_df['Max Temp'] < 85]
max_temp

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,42.55,68,7,4.00,US,1702325191
1,1,carnarvon,-24.8667,113.6333,74.86,58,0,7.29,AU,1702325191
2,2,papatowai,-46.5619,169.4708,47.79,70,54,19.91,NZ,1702325192
3,3,lakheri,25.6667,76.1667,61.57,45,0,2.93,IN,1702325192
4,4,afaahiti,-17.7500,-149.2833,82.27,80,92,22.41,PF,1702325192
...,...,...,...,...,...,...,...,...,...,...
575,575,qurayyat,31.3318,37.3428,55.58,88,80,6.91,SA,1702325336
576,576,tomaszow lubelski,50.4477,23.4162,35.13,98,100,14.18,PL,1702325336
579,579,kailua-kona,19.6406,-155.9956,84.27,72,20,4.61,US,1702325337
580,580,kamianske,48.5113,34.6021,35.64,100,100,5.10,UA,1702325233


In [5]:
# Narrow down cities that fit criteria and drop any
max_temp = city_data_df['Max Temp'] < 85
cloudiness = city_data_df['Cloudiness'] > 30

perfect_weather = city_data_df[max_temp & cloudiness]

# Drop any rows with null values
perfect_weather_df = perfect_weather.dropna()

# Display sample data
perfect_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,papatowai,-46.5619,169.4708,47.79,70,54,19.91,NZ,1702325192
4,4,afaahiti,-17.7500,-149.2833,82.27,80,92,22.41,PF,1702325192
5,5,san antonio de pale,-1.4014,5.6325,77.13,88,100,4.81,GQ,1702325095
6,6,waitangi,-43.9535,-176.5597,57.81,88,100,8.01,NZ,1702325192
7,7,carutapera,-1.1950,-46.0200,83.71,69,35,10.49,BR,1702325193
...,...,...,...,...,...,...,...,...,...,...
573,573,wewak,-3.5534,143.6268,77.94,90,99,5.57,PG,1702325336
574,574,lospalos,-8.5186,127.0025,74.77,87,100,3.02,TL,1702325336
575,575,qurayyat,31.3318,37.3428,55.58,88,80,6.91,SA,1702325336
576,576,tomaszow lubelski,50.4477,23.4162,35.13,98,100,14.18,PL,1702325336


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_weather_df[['City','Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,papatowai,NZ,-46.5619,169.4708,70,
4,afaahiti,PF,-17.7500,-149.2833,80,
5,san antonio de pale,GQ,-1.4014,5.6325,88,
6,waitangi,NZ,-43.9535,-176.5597,88,
7,carutapera,BR,-1.1950,-46.0200,69,
...,...,...,...,...,...,...
573,wewak,PG,-3.5534,143.6268,90,
574,lospalos,TL,-8.5186,127.0025,87,
575,qurayyat,SA,31.3318,37.3428,88,
576,tomaszow lubelski,PL,50.4477,23.4162,98,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000

params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
papatowai - nearest hotel: No hotel found
afaahiti - nearest hotel: Omati Lodge
san antonio de pale - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
carutapera - nearest hotel: No hotel found
bamboo flat - nearest hotel: Hotel Megapode Nest
grytviken - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
jamestown - nearest hotel: DoubleTree Jamestown
susuman - nearest hotel: No hotel found
kendari - nearest hotel: City Hotel Kendari
edinburgh of the seven seas - nearest hotel: No hotel found
uyuni - nearest hotel: Hostel Jerian
bozoum - nearest hotel: No hotel found
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
namibe - nearest hotel: Hotel Chik Chik Namibe
sabang - nearest hotel: Sabang Hill
ko pha ngan - nearest hotel: The Shore Resort
whitehorse - nearest hotel: Town & Mountain Hotel
port-aux-francais - nearest hotel: Keravel
pandan - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hot

,City,Country,Lat,Lng,Humidity,Hotel Name
2,papatowai,NZ,-46.5619,169.4708,70,No hotel found
4,afaahiti,PF,-17.7500,-149.2833,80,Omati Lodge
5,san antonio de pale,GQ,-1.4014,5.6325,88,No hotel found
6,waitangi,NZ,-43.9535,-176.5597,88,Hotel Chathams
7,carutapera,BR,-1.1950,-46.0200,69,No hotel found
...,...,...,...,...,...,...
573,wewak,PG,-3.5534,143.6268,90,Village Inn
574,lospalos,TL,-8.5186,127.0025,87,Hotel Roberto Carlos
575,qurayyat,SA,31.3318,37.3428,88,دانة الشمال
576,tomaszow lubelski,PL,50.4477,23.4162,98,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    size='Humidity',
    cmap='Viridis',
    colorbar=True,
    color = 'City',
    legend=True,
    hover_cols=['Hotel Name', 'Country'],
    tiles='OSM')


# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)